# Pre-processing

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('data/spotify_songs.csv')

In [ ]:
display(data)
display(data.describe().T.style.background_gradient(cmap='YlGnBu'))

In [ ]:
# check for missing values
missing_values = data.isnull().sum()
display(missing_values)

# display the lines with missing values
missing_data = data[data.isnull().any(axis=1)]
display(missing_data)

In [ ]:
# Remove id columns
data = data.drop(columns=['track_id', 'track_album_id', "playlist_id"], axis=1)
# drop the missing values
data = data.dropna()
display(data)


In [ ]:
# transform track_album_release_date into datetime
data['track_album_release_date'] = pd.to_datetime(data['track_album_release_date'], format='mixed')

# transform categorical columns into categorical data type
categorical_cols = ['playlist_genre', 'playlist_subgenre', 'track_artist', 'playlist_name', 'track_album_name']
for col in categorical_cols:
    data[col] = data[col].astype('category')

# transform the duration_ms into minutes
data['duration_s'] = data['duration_ms'] / 1000
data.drop(columns=['duration_ms'], inplace=True)

# For numeric columns that represent discrete values (like key and mode), convert to categorical
key_mapping = {
    0: 'C', 1: 'C♯/D♭', 2: 'D', 3: 'D♯/E♭', 4: 'E', 5: 'F',
    6: 'F♯/G♭', 7: 'G', 8: 'G♯/A♭', 9: 'A', 10: 'A♯/B♭', 11: 'B'
}
data['key'] = data['key'].map(key_mapping).astype('category')

data['mode'] = data['mode'].map({0: 'Minor', 1: 'Major'}).astype('category')



In [ ]:
# check for duplicates
duplicates = data.duplicated().sum()
display(duplicates)

In [ ]:
data.info()

# Descriptive Analysis

# 2. Réduction de dimension par ACP (Analyse en Composantes Principales)
Dans cette partie, nous allons effectuer une analyse en composantes principales (ACP) sur les données prétraitées. L'ACP est une technique de réduction de dimension qui permet de projeter les données d'origine dans un espace de dimension inférieure.
Nous avons gardé 20 variables et nous allons étudier s'il est possible de réduire la dimensionnalité de ces données tout en préservant un maximum d'information.

In [ ]:
import numpy as np 

import matplotlib.pyplot as plt
import seaborn as sns

### 2.1 Format des données
Ici on ne sélectionne que les variables quantitatives, en y ajoutant une variable qualitative (`playlist_genre`) pour voir si elle a un impact sur la projection des données. On garde au final 11 variables quantitatives et 1 variable qualitative.

On décide de normaliser les données pour que chaque variable ait une moyenne de 0 et un écart-type de 1. Cela est important car l'ACP est sensible à l'échelle des variables. On utilise la méthode `StandardScaler` de `sklearn` pour normaliser les données.

In [ ]:
# Select only the quantitative columns

qualisup = 'playlist_genre'

data_quanti = data.select_dtypes(include=['int64', 'float64'])
data_quanti[qualisup] = data[qualisup]
data_quanti = data_quanti.set_index(qualisup)

data_quanti.info()

In [ ]:
data_quanti[:5]

In [ ]:
from sklearn.decomposition import PCA
import prince 
from sklearn.preprocessing import StandardScaler

normalize_bool = True

if normalize_bool:
    scaler = StandardScaler()
    data_scaled = scaler.fit_transform(data_quanti)
else:
    data_scaled = data_quanti.values

pca = PCA(
    n_components=10,  # Number of components to keep
    random_state=1  # For reproducibility
)

projected = pca.fit_transform(data_scaled)

explained_variance = pca.explained_variance_ratio_

eig = pd.DataFrame(
    {
        "Dimension" : [f"PC{i+1}" for i in range(len(explained_variance))],
        "Variance" : np.round(pca.explained_variance_, 2),
        "% explained variance" : np.round(explained_variance*100, 1),
        "% cumulative variance" : np.round(np.cumsum(explained_variance)*100, 1)
    }
)
eig

In [ ]:
fig = plt.figure(figsize=(15,8))
ax = fig.add_subplot(1,2,1)
ax.bar(range(10), pca.explained_variance_ratio_[:10]*100, align='center',
        color='coral', ecolor='black')
ax.set_xticks(range(10))
ax.set_ylabel("Variance")
ax.set_title("", fontsize=35)
ax.set_title(u"Pourcentage de variance expliqué", fontsize=20)

ax = fig.add_subplot(1,2,2)
ax.plot(np.cumsum(pca.explained_variance_ratio_), color='coral', marker='o')
ax.hlines(0.80, 0, 10, colors='grey', linestyles='dashed', alpha=0.5)
ax.set_title(u'Pourcentage de variance expliqué cumulé', fontsize=20)

fig.suptitle(u"Résultat ACP", fontsize=25)
plt.show()

**Interprétation :** 

L’analyse de la variance expliquée montre que les **7 premières composantes principales** permettent de **représenter 80,1 % de la variance totale** du jeu de données. Cela signifie que l’essentiel de l’information contenue dans les **11 variables numériques initiales** (comme *danceability*, *energy*, *speechiness*, *tempo*, etc.) peut être résumé avec seulement 7 dimensions, ce qui représente une **réduction significative de la complexité** du dataset tout en conservant une bonne qualité descriptive.

Dans cette optique de réduction de dimension, il serait **pertinent de conserver ces 7 composantes principales** pour la suite des analyses (clustering, visualisation, classification), car elles capturent la structure principale des données tout en éliminant le "bruit".

Dans l’analyse factorielle, nous avons choisi d’**interpréter les trois premières composantes principales**, qui à elles seules expliquent 45 % de la variance totale. Elles offrent un bon compromis entre lisibilité et pertinence pour une visualisation ou une première analyse des relations entre les variables et les genres musicaux (*playlist_genre*).

In [ ]:
fig = plt.figure(figsize=(12,6))
box=plt.boxplot(projected[:,0:3],whis=100)
plt.title(u"Distribution des premières composantes", fontsize=20)
plt.show()

**Interprétation :**
- Le graphique ci-dessus montre la distribution des premières composantes principales. 

In [ ]:
pca_prince = prince.PCA(
    n_components=3,
    n_iter=3,
    rescale_with_mean=normalize_bool,
    rescale_with_std=normalize_bool,
    copy=True,
    check_input=True,
    engine='sklearn',
    random_state=1
)
pca_prince = pca_prince.fit(
    data_quanti,
    sample_weight=None,
    column_weight=None,
    supplementary_columns=None
)

# Create a correlation matrix of the PCA compnenets
correlations = pca_prince.column_correlations
plt.figure(figsize=(10, 8))
sns.heatmap(correlations, annot=True, cmap='RdBu', center=0)
plt.title('Corrélations entre les variables et les composantes principales')
plt.show()


In [ ]:
# Récupération des coordonnées des variables (correlations avec les composantes principales)
coords = pca_prince.column_correlations

# Création de la figure et des sous-graphiques
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Boucle pour créer les trois graphiques
for idx, (ax, (x_comp, y_comp)) in enumerate(zip(axes, [(0, 1), (1, 2), (0, 2)])):
    ax.grid(False)
    for i in range(coords.shape[0]):
        ax.arrow(0, 0, coords.iloc[i, x_comp], coords.iloc[i, y_comp], 
                 color='black', alpha=0.7, head_width=0.02, head_length=0.03)
        ax.text(coords.iloc[i, x_comp] * 1.1, coords.iloc[i, y_comp] * 1.1, 
                coords.index[i], color='black', ha='center', va='center')

    # Ajout du cercle de rayon 1
    ax.add_artist(plt.Circle((0, 0), radius=1, color='cornflowerblue', fill=False))

    # Ajout des axes
    ax.axhline(0, color='gray', linestyle='--', linewidth=0.5)
    ax.axvline(0, color='gray', linestyle='--', linewidth=0.5)

    # Paramètres du graphique
    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel(f'Composante principale {x_comp + 1}')
    ax.set_ylabel(f'Composante principale {y_comp + 1}')
    ax.set_title(f'Projection des variables: CP{x_comp + 1} vs CP{y_comp + 1}')

plt.tight_layout()
plt.show()

La table des correlations et les trois graphiques ci-dessus représentent les projections des features sur les trois premières composantes principales nous donne des informations sur la structure des données réduites.

- **Composante principale 1** : 
    - Les variables `energy (-0.91)`, `loudness (-0.80)` et `acousticness (+0.72)` sont linéairement corrélées avec la première composante principale, en soulignant que `energy` et `loudness` sont inversément corrélées avec `acousticness`. Cela indique que la CP1 oppose les morceaux **énergiques, forts en volume et peu acoustiques** (ex : rock, électro) aux morceaux **calmes, acoustiques et peu énergétiques** (ex : folk, classique).
- **Composante principale 2** : Sur le graphique de gauche, on remarque une opposition des variables `instrumentalness (+0.45)`, `duration_s (+0.38)` contre `danceability (-0.68)`, `valence (-0.62)`, `track_popularity (-0.37)`, `speechiness (-0.39)`. Cette composante principale oppose deux profils de morceaux :
    - D’un côté, les morceaux **instrumentaux, longs et peu populaires** (forte contribution de `instrumentalness` et `duration_s`), souvent associés à des genres comme le classique ou le jazz.
    - De l’autre, les chansons **courtes, dansantes, joyeuses et populaires** (forte contribution de `danceability`, `valence` et `track_popularity`), typiques de la pop ou de la musique de club. 
    - Enfin, cette opposition suggère que les morceaux avec des paroles marquées (`speechiness`) et une structure rythmique engageante (`danceability`) sont plus susceptibles de générer de la popularité.
- **Composantes principales 2 et 3** : Sur le graphique au centre, on peut extraire plusieurs informations :
    - Les morceaux dansants et joyeux (haute valence) s'opposent dans une moindre mesure aux morceaux de faible tempo. De plus, ces morceaux semblent avoir peu de versions live.
    - On observe aussi que les morceaux instrumentaux et longs ont généralement une faible popularité, tandis que les morceaux courts et peu instrumentaux sont souvent plus populaires, ce qui est typique de la musique pop, qui est souvent plus accessible et commerciale.
- **Composante principale 3** : La troisième composante (CP3) révèle un paradoxe : elle regroupe des morceaux à fort potentiel dansant (`danceability`) et mood positif (`valence`), mais qui restent peu populaires (`track_popularity`). Ces morceaux sont souvent instrumentaux (`instrumentalness`), longs (`duration_s`) et à tempo faible (`tempo`), ce qui les éloigne des standards des charts. Cette composante pourrait représenter des créations artistiques équilibrant danse et complexité, mais peinant à atteindre un large public.

Pour mieux comprendre à quoi correspond les type morceaux extraits par ces composantes principales, nous allons regarder les morceaux (individus) contribuant le plus à chacune des composantes principales.

In [ ]:
# display la mediane de notre dataset quantitatif à fins de comparaison
median_values = data_quanti.median()
median_values = median_values.to_frame(name='median')
median_values = median_values.reset_index()
median_values.columns = ['feature', 'median']
median_values['feature'] = median_values['feature'].astype('category')
median_values = median_values.set_index('feature')
median_values = median_values.sort_index()
median_values = median_values.T
median_values

In [ ]:
# Calculate the contributions of individuals to the first principal component
contributions = projected[:, 0]

# Get the indices of the top 5 positive contributors
top_5_positive_indices = np.argsort(contributions)[-5:]

# Get the indices of the top 5 negative contributors
top_5_negative_indices = np.argsort(contributions)[:5]

# Extract the corresponding rows from the original dataset for positive contributions
top_5_positive_tracks = data_quanti.iloc[top_5_positive_indices].copy()
top_5_positive_tracks['track_name'] = data.iloc[top_5_positive_indices]['track_name'].values
top_5_positive_tracks['track_artist'] = data.iloc[top_5_positive_indices]['track_artist'].values
top_5_positive_tracks['contribution'] = contributions[top_5_positive_indices]

# Extract the corresponding rows from the original dataset for negative contributions
top_5_negative_tracks = data_quanti.iloc[top_5_negative_indices].copy()
top_5_negative_tracks['track_name'] = data.iloc[top_5_negative_indices]['track_name'].values
top_5_negative_tracks['track_artist'] = data.iloc[top_5_negative_indices]['track_artist'].values
top_5_negative_tracks['contribution'] = contributions[top_5_negative_indices]

# Display the top 5 positive and negative tracks with additional information
print("Top 5 Positive Contributions (PC1):")
display(top_5_positive_tracks)

print("Top 5 Negative Contributions (PC1) :")
display(top_5_negative_tracks)

### Interprétation de l'Analyse en Composantes Principales (PCA)


#### **Analyse des principales corrélations :**
- **PC1 (Composante Principale 1) :**  
  - **Corrélations négatives fortes :** `energy (-0.91)`, `loudness (-0.80)`.  
    → Cette composante sépare les morceaux énergiques et bruyants des morceaux plus doux.  
  - **Corrélation positive forte :** `acousticness (+0.72)`.  
    → Elle oppose également les morceaux acoustiques aux morceaux énergétiques.  

- **PC2 (Composante Principale 2) :**  
  - **Corrélations négatives :** `danceability (-0.68)`, `valence (-0.62)`, `track_popularity (-0.37)`, `speechiness (-0.39)`.  
    → Cette composante semble regrouper les morceaux dansants
  - **Corrélations positives :** `instrumentalness (+0.45)`, `duration_s (+0.38)`.  
    → Elle représente les morceaux plus longs et instrumentaux.  
  → Cette dimension est donc liée à l'humeur des morceaux, opposant les morceaux joyeux et dansants aux morceaux plus sombres et instrumentaux.

- **PC3 (Composante Principale 3) :**  
  - **Corrélations positives :** `danceability (+0.46)`, `instrumentalness (+0.39)`, `duration_s (+0.45)`.  
    → Cette dimension semble être liée à la dansabilité et à la durée des morceaux.  
  - **Corrélation négative :** `tempo (-0.35)`.  
    → Elle oppose les morceaux rapides et lents.  

- **PC4 (Composante Principale 4) :**  
  - **Corrélation positive forte :** `speechiness (+0.71)`.  
    → Cette composante est dominée par la présence de paroles (ex : rap).  
  - **Corrélation positive :** `liveness (+0.58)`.  
    → Elle met en avant les morceaux enregistrés en live.  

### **Interprétation globale :**
- **PC1 sépare les morceaux énergiques et bruyants des morceaux acoustiques.**  
- **PC2 distingue les morceaux dansants et joyeux des morceaux instrumentaux et longs.**  
- **PC3 différencie les morceaux en fonction de leur dansabilité et de leur tempo.**  
- **PC4 met en avant les morceaux avec beaucoup de paroles et ceux enregistrés en live.**  



In [ ]:
# %load solutions/pmca/plot_pca.py
def plot_pca(ax1=0, ax2=1, pca=pca, data=data_quanti):
  dataset = pca.transform(data)
  dataset.reset_index(inplace=True)
  sns.scatterplot(data = dataset,
                  x = ax1, y = ax2,
                  hue = qualisup, alpha=.3)

  plt.xlabel('Component {} — {:.2f}%'.format(ax1, pca.percentage_of_variance_[ax1]))
  plt.ylabel('Component {} — {:.2f}%'.format(ax2, pca.percentage_of_variance_[ax2]))
  plt.grid(True)
  plt.show()

plot_pca(0,1)
plot_pca(1,2)
plot_pca(0,2)